In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f7e801ce5a96c399bcfc8a7fd8d0f291fb372e4f4c4926cb3ae5a9186b2408e8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Pratices").getOrCreate()

In [6]:
spark

In [8]:
#Read the dataset
training = spark.read.csv('test1.csv',header=True,inferSchema = True)

In [9]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [10]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [15]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','Experience'],outputCol = 'Independent features')

In [16]:
output = featureassembler.transform(training)

In [17]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [18]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [19]:
finalized_data = output.select('Independent features','Salary')

In [20]:
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [22]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [23]:
### Coefficients
regressor.coefficients

DenseVector([1771.9298, -298.2456])

In [24]:
### Intercepts
regressor.intercept

-21912.280701753818

In [25]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [26]:
pred_results.predictions.show()


+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [29.0,4.0]| 20000|28280.701754385842|
|          [30.0,8.0]| 25000|28859.649122806983|
+--------------------+------+------------------+



In [27]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError


(6070.175438596412, 41733456.44813674)